In [4]:
!pip install --upgrade scikit-learn
!pip install keras
!pip install tensorflow
!pip install joblib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
import joblib


# Set the paths for the diseased and non-diseased brain images
diseased_path = "minidataset/diseased/train"
non_diseased_path = "minidataset/healthy/train"
# Set the number of images to use for training
num_diseased = 200
num_non_diseased = 100

# Initialize the feature vectors and labels
X_train = []
y_train = []
# Load the diseased brain images and extract features
def extract_features(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (200, 200))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), transform_sqrt=True)
    return features

for i, filename in enumerate(os.listdir(diseased_path)[:num_diseased]):
    img_path = os.path.join(diseased_path, filename)
    features=extract_features(img_path)
    X_train.append(features)
    y_train.append(1)

# Load the non-diseased brain images and extract features
for i, filename in enumerate(os.listdir(non_diseased_path)[:num_non_diseased]):
    img_path = os.path.join(non_diseased_path, filename)
    features=extract_features(img_path)
    X_train.append(features)
    y_train.append(0)
# Train a linear SVM classifier
clf = LinearSVC()
clf.fit(X_train, y_train)

joblib.dump(clf, 'svm_model.joblib')  


C:\Users\sweth\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


['svm_model.joblib']

In [3]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

from keras.models import load_model
model = joblib.load('svm_model.joblib')


# Define a function to open a file dialog to select a new image
def choose_file():
    global selected_file_path
    selected_file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if selected_file_path:
        preview_image(selected_file_path)
        

# Define a function to preview the selected image
def preview_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (200, 200))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = ImageTk.PhotoImage(img)
    image_label.config(image=img)
    image_label.image = img
    
    
    
#Define a function to classify the selected image
def classify_image():
    if not selected_file_path:
        messagebox.showerror("Error", "No image selected") 
        return 
    pred_label = predict_image(selected_file_path)
    if pred_label == 1:
        messagebox.showinfo("Result", "The image is diseased.")
    else:
        messagebox.showinfo("Result", "The image is not diseased.")

        
# Define a function to predict the label of a new image
def predict_image(image_path):
    features = extract_features(image_path)
    pred_label = model.predict([features])[0]
    return pred_label


# Define a function to extract features from a single image
def extract_features(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (200, 200))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), transform_sqrt=True)
    return features


#Create the main window and widgets
window = tk.Tk()
window.title("Brain Disease Classifier")
window.geometry("600x400")
window.resizable(False, False)

title_label = tk.Label(window, text="Brain Disease Classifier", font=("Helvetica", 16), pady=10)
title_label.pack()

#Create the buttons to load an image and classify it
load_button = tk.Button(window, text="Load Image", command=choose_file, padx=10, pady=5)
load_button.pack()

classify_button = tk.Button(window, text="Classify", command=classify_image, padx=10, pady=5)
classify_button.pack()

#Create a label to display the selected image
image_label = tk.Label(window)
image_label.pack()

#Start the GUI event loop
window.mainloop()
